In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import proplot as pplt
from scipy.fftpack import * 

2021.08.08

尝试使用proplot绘制平均图

温度观测状况不变，CN05.1只在中国大陆有分析结果，仍然使用原本的数据

2022.02.13

计算MJJA的平均态，并且绘制气温的最大、最小、平均的状态

使用PD制表，写出CSV表格说明平均态的统计结果

## 数据读入

- 直接获取此前使用1hr间隔计算的tmax tmin tmean 使用路径：

dir_in = "/raid52/yycheng/MPAS/REFERENCE/TEMP_DATA/mask_t2m/mask_res/"

- 使用三小时取样计算的t2m

dir_in = "/raid52/yycheng/MPAS/REFERENCE/TEMP_DATA/mask_t2m/mask_res_3hr/"

- 使用六小时取样计算t2m

dir_in = "/raid52/yycheng/MPAS/REFERENCE/TEMP_DATA/mask_t2m/mask_res_6hr/"

最后进行混合使用 mean 使用 6HR取样计算 min max使用1HR计算

In [ ]:
var = {} # 放置所有变量的字典
# read mean
dir_in = "/raid52/yycheng/MPAS/REFERENCE/TEMP_DATA/mask_t2m/mask_res/"
filename_obs  = "mask_sel_CN05.1_Tm_1961_2018_daily_025x025.nc"
filename_vr     = "mask_mean_t2m_98-17_VR.nc"
filename_rcm    = "mask_mean_t2m_98-17_RCM.nc"
ds_or = {}
ds_or['obs'] = xr.open_dataset(dir_in + filename_obs)
ds_or['vr']     = xr.open_dataset(dir_in + filename_vr)
ds_or['rcm']    = xr.open_dataset(dir_in + filename_rcm  )
var['mean'] = {}
var['mean']['obs'] = ds_or['obs']['tm']#[:,  :, :]
var['mean']['vr'] = ds_or['vr']['t2m'] - 273.15
var['mean']['rcm'] = ds_or['rcm']['t2m'] - 273.15

# 2022.02.15 min max切换到六小时的计算结果
dir_in = "/raid52/yycheng/MPAS/REFERENCE/TEMP_DATA/mask_t2m/mask_res_6hr/"

# max
filename_obs  = "mask_sel_CN05.1_Tmax_1961_2018_daily_025x025.nc"
filename_vr     = "mask_max_t2m_98-17_VR.nc"
filename_rcm    = "mask_max_t2m_98-17_RCM.nc"
ds_or = {}
ds_or['obs'] = xr.open_dataset(dir_in + filename_obs)
ds_or['vr']     = xr.open_dataset(dir_in + filename_vr)
ds_or['rcm']    = xr.open_dataset(dir_in + filename_rcm  )
var['max'] = {}
var['max']['obs'] = ds_or['obs']['tmax']#[:,  :, :]
var['max']['vr'] = ds_or['vr']['t2m'] - 273.15
var['max']['rcm'] = ds_or['rcm']['t2m'] - 273.15

# min
filename_obs  = "mask_sel_CN05.1_Tmin_1961_2018_daily_025x025.nc"
filename_vr     = "mask_min_t2m_98-17_VR.nc"
filename_rcm    = "mask_min_t2m_98-17_RCM.nc"
ds_or = {}
ds_or['obs'] = xr.open_dataset(dir_in + filename_obs)
ds_or['vr']     = xr.open_dataset(dir_in + filename_vr)
ds_or['rcm']    = xr.open_dataset(dir_in + filename_rcm  )
var['min'] = {}
var['min']['obs'] = ds_or['obs']['tmin']#[:,  :, :]
var['min']['vr'] = ds_or['vr']['t2m'] - 273.15
var['min']['rcm'] = ds_or['rcm']['t2m'] - 273.15

# change coords
var_list = ['obs', 'vr', 'rcm']
for i in ['mean', 'max', 'min']:
    for j in var_list:
        rename_dict = dict(zip(var[i][j].coords.keys(), var[i]['obs'].coords.keys()))
        # for rename_i in rename_dict:
            # print(rename_i + " -----converting to----- " + rename_dict[rename_i])

        var[i][j] = var[i][j].rename(rename_dict)
        var[i][j]._coords = var[i]['obs']._coords
        var[i][j] = var[i][j].rename(j)


## 计算平均态
调整到MJJA

In [ ]:
var_selmonth = {}
var_selmonth['mjja'] = {}
for var_name in ['mean', 'max', 'min']:
    var_selmonth['mjja'][var_name] = {}
    for mod_name in ['obs', 'vr', 'rcm']:
        time_idx_mjja = var[var_name][mod_name].time.dt.month.isin([5,6,7,8])
        var_selmonth['mjja'][var_name][mod_name] = var[var_name][mod_name].isel(time = time_idx_mjja).mean(dim = 'time')

## 计算指标

### 计算制图表格

In [ ]:
def nested_dict():
    from collections import defaultdict
    return defaultdict(nested_dict)
# 进行mask，并且整理成1D数组，得到1D-pattern，用于后续的计算
# 计算指数的平均值
var_1d = nested_dict()
for iseason in ['mjja']:
    for var_name in ['mean', 'max', 'min']:
        for mod_name in ['obs', 'vr', 'rcm']:
            # 获取1D的平均态 并以RCM为基础进行NAN的MASK
            rcm_not_nan = ( ~np.isnan(var_selmonth['mjja']['mean']['rcm'])) &\
                 (~np.isnan(var_selmonth['mjja']['mean']['obs'])) &\
                 (~np.isnan(var_selmonth['mjja']['mean']['vr']) )
            var_1d_temp = xr.where(rcm_not_nan, var_selmonth[iseason][var_name][mod_name], np.nan).values.ravel()
            var_1d[iseason][var_name][mod_name] = var_1d_temp[~np.isnan(var_1d_temp)]
            # shape check
            # print(var_1d[iseason][var_name][mod_name].shape)

In [ ]:
import skill_metrics as sm
# 计算泰勒图诸要素 到字典taylor_ts中

# 进行mask，并且整理成1D数组，得到1D-pattern，用于后续的计算
# 计算指数的平均值
taylor_space = nested_dict()
for iseason in ['mjja']:
    for var_name in ['mean', 'max', 'min']:
        for mod_name in ['obs', 'vr', 'rcm']:
            temp_obs = var_1d[iseason][var_name]['obs']
            temp_mod = var_1d[iseason][var_name][mod_name]
            # taylor count
            taylor_space[iseason][var_name][mod_name] = sm.taylor_statistics(temp_mod, temp_obs)
            # normalized 注意先后顺序，先归一化CRMSD，再SDEV，否则用归一化后的SDEV计算RMSD会出现问题
            # taylor_space[iseason][var_name][mod_name]['crmsd'] = taylor_space[iseason][var_name][mod_name]['crmsd'] / taylor_space[iseason][var_name][mod_name]['sdev'][0]
            # taylor_space[iseason][var_name][mod_name]['sdev'] = taylor_space[iseason][var_name][mod_name]['sdev'] / taylor_space[iseason][var_name][mod_name]['sdev'][0]

### 合并表格并写出csv文件

In [ ]:
pd_templist = []
ind = np.array([0]) # index的整数
for iseason in ['mjja']:
    for var_name in ['mean', 'max', 'min']:
        for mod_name in ['obs', 'vr', 'rcm']:
            temp_ccoef = taylor_space[iseason][var_name][mod_name]['ccoef'][1]
            temp_crmsd = taylor_space[iseason][var_name][mod_name]['crmsd'][1]
            temp_sdev  = taylor_space[iseason][var_name][mod_name]['sdev'][1]
            temp_mean  = var_1d[iseason][var_name][mod_name].mean()
            # temp_bias  = var_1d[iseason][var_name][mod_name].mean() - var_1d[iseason][var_name]['obs'].mean()
            temp_bias  = sm.bias(var_1d[iseason][var_name][mod_name], var_1d[iseason][var_name]['obs'])
            temp_rmsd  = sm.rmsd(var_1d[iseason][var_name][mod_name], var_1d[iseason][var_name]['obs'])
            
            pd_templist.append( pd.DataFrame(data = [[var_name, mod_name, temp_mean, temp_bias, temp_ccoef, temp_crmsd, temp_rmsd, temp_sdev]], \
                columns = ['variable','model','mean', 'bias','ccoef','crmsd','rmsd','sdev'], index=  ind) )
            ind = ind + 1

# 合并表格并写出
table_pdconcat = pd.concat(pd_templist)
table_pdconcat.to_csv("./output_table/t2m_SpatialTaylor.2022.02.15.csv")

## 绘图部分

In [ ]:
# 国内政区图的绘制
# Load the border data, CN-border-La.dat is download from
# https://gmt-china.org/data/CN-border-La.dat
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.patches as mpatches
import cmaps

cn_border_file = "/m2data2/yycheng/data_stage/CN-border/CN-border_line/CN-border-La.dat"
with open(cn_border_file) as src:
    context = src.read()
    blocks = [cnt for cnt in context.split('>') if len(cnt) > 0]
    borders = [np.fromstring(block, dtype=float, sep=' ') for block in blocks]


### 画图renew
#### 2021.08.08
cn_border_file + shapefile ，但是存在重叠，尝试消除掉China边界，但是其他邻国边界无法处理；
#### 2021.08.09
不使用cn_border_file ，使用shapefile + coast_line（proplot自带） 的办法
shapefile有一些重叠，不绘制行政区
shapefile重新进行绘制，考虑来自 domain_info 中测试的多个shape file中挑选出地资所（改变了prj方式，具体查看prj后缀文件）进行使用

In [ ]:
def border_plot(axs):
    """
    进行行政区划的绘制，通过shapefilereader绘制存档的shp文件，需要传入axs，并逐个绘制
    比较消耗时间，调整完毕后最后添加边界的绘制
    """
    ##---- 直接绘图，从边界文件添加
    # for ax_ind in axs:
    # for line in borders:
    #     axs.plot(line[0::100], line[1::100], lw = 0.5, color='gray',transform=ccrs.Geodetic())
    #     axs.plot(line[0::10], line[1::10], lw = 0.4, color='black',transform=ccrs.Geodetic())
    ##---- 使用shp文件添加
        # shapefile数据下载的位置：
    # http://gaohr.win/site/blogs/2017/2017-04-18-GIS-basic-data-of-China.html
    world_border_shapefile = "/m2data2/yycheng/data_stage/CN-border/World/country.shp"
    river_border_shapefile =  "/raid52/yycheng/MPAS/REFERENCE/MODEL_CONSTANT/R1/" + "hyd1_4l.shp"
    southsea_shapefile     = "/m2data2/yycheng/data_stage/CN-border/SouthSea/" + "southsea_island.shp"
    ninelines_shapefile     = "/m2data2/yycheng/data_stage/CN-border/SouthSea/" + "nine_lines.shp"
    ## 来源： 沛沛的诸省 + 诸岛
    bou24p_shapefile     = "/m2data2/yycheng/data_stage/CN-border/peipeihelp/" + "bou2_4p.shp"
    ## 来源： https://www.resdc.cn/data.aspx?DATAID=200
    province_shapefile     = "/m2data2/yycheng/data_stage/CN-border/CN-sheng/" + "change_proj_CN-sheng-A.shp"

    for ax in axs:
        # world     = shpreader.Reader(world_border_shapefile).geometries()
        # river     = shpreader.Reader(river_border_shapefile).geometries()
        river     = shpreader.Reader(river_border_shapefile, encoding = "gbk")
        # bou24p    = shpreader.Reader(bou24p_shapefile).geometries()
        ninelines = shpreader.Reader(ninelines_shapefile).geometries()
        province  = shpreader.Reader(province_shapefile).geometries()
        # ax.add_geometries(river, ccrs.PlateCarree(), facecolor='none', edgecolor='b', linewidth=0.4, zorder=1)
        # ax.add_geometries(world, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.4, zorder=1)
        # ax.add_geometries(bou24p, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.6, zorder=1) # 沛沛map
        ax.add_geometries(province, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.6, zorder=1) # 地资所
        ax.add_geometries(ninelines, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.6, zorder=1)
        # 绘制部分的shapefile
        for region in river.records():
            if (region.attributes['NAME'] in ['黄河','长江']):
                # print("----- draw river! -----")
                # 此处需要使用 [] 让region.geometry可以迭代
                ax.add_geometries([region.geometry], ccrs.PlateCarree(), facecolor='none', edgecolor='b', linewidth=0.4, zorder=1)


In [ ]:
def add_southchinasea(axs):
    '''
    用于制作南海小图的axes，偷懒使用上面绘制的borders绘制地图的边界状况，视情况准备其他边界数据
    需要特别对新axes进行绘制
    '''
    for axs_single in axs:
        axs_small = axs_single.inset_axes(bounds = (0.85, 0.02, 0.15, 0.35), transform = 'axes', proj = 'cyl', zoom = False)

        axs_small.format(
        lonlim=(107, 123), latlim=(0, 25),
        #----- 地图底图设置 -----
        reso = 'med',
        coast = True,
        coastlinewidth = 0.4,
        borders = False,
        lakes = False,
        land  = False,
        ocean = False,
        labels = False,
        longrid  = False,
        latgrid  = False,
        #-----GEO axis-----
        )
        # 自己设置的绘制中国区域的border_plot
        border_plot([axs_small])
        return axs_small

In [ ]:
# import proplot as plot
from matplotlib import pyplot as plt
import proplot as plot
import collections
# ----- get filter vars coords-----

lon = var_selmonth['mjja']['mean']['vr'].lon.values
lat = var_selmonth['mjja']['mean']['vr'].lat.values

#----- change font -----
pplt.rc['font.serif'] = 'Arial'

#----- create plot -----
# fig, axs = plot.subplots(ncols=3 ,nrows=3, proj=('cyl'), journal = 'agu2', share = True)
fig, axs = plot.subplots(ncols=3 ,nrows=3, proj=('cyl'), figwidth = '190mm', figheight = '150mm', share = True, tight = True)
# fig, axs = plot.subplots(ncols=3 ,nrows=3, proj=('cyl'), share = True, hspace = (0,0), wspace = (0,0))
# fig, axs = plot.subplots(ncols=3 ,nrows=3, proj=('cyl'))
m_contour_list = [] # 用于保存contour设置，后续设置colorbar使用

#----- 添加海洋以及行政区划 -----
border_plot(axs)

def nested_dict():
    return collections.defaultdict(nested_dict)
mcontourf_dict = nested_dict()
#----- colorbar ticks 统一设置 -----
cmap_mean      = cmaps.BkBlAqGrYeOrReViWh200
# cmap_diff = cmaps.cmp_flux 
cmap_diff = cmaps.ncl_default 

# mean_ticks = np.concatenate((np.linspace(0,12,13), np.linspace(14,34,6)), axis=0)
mean_ticks = np.linspace(-5,35,int(40/1) + 1)
diff_ticks = np.linspace(-5, 5, 21)
print("----- tick levels is : " + str(mean_ticks))

#----- plot contourf and titile axs ------

###----- main plot -----
plot_ind = 0
for var_name in ['mean', 'max', 'min']:
    for mod_name in ['obs','vr','rcm']:
        axs_small_temp = add_southchinasea([axs[plot_ind]]) # 生成南海小图的axs
        if (mod_name == 'obs'):
            mcontourf_dict['mjja'][var_name][mod_name] = axs[plot_ind].contourf(lon, lat, var_selmonth['mjja'][var_name][mod_name].values,\
                levels = mean_ticks, cmap = cmap_mean, extend = 'both')
            # 添加南海小图的等值线绘制，方法较为原始，直接赋值上文
            axs_small_temp.contourf(lon, lat, var_selmonth['mjja'][var_name][mod_name].values,\
            levels = mean_ticks, cmap = cmap_mean, extend = 'both')
        else:
            mcontourf_dict['mjja'][var_name][mod_name] = axs[plot_ind].contourf(lon, lat, var_selmonth['mjja'][var_name][mod_name].values - var_selmonth['mjja'][var_name]['obs'].values,\
                levels = diff_ticks, cmap = cmap_diff, extend = 'both')
            # 添加南海小图的等值线绘制，方法较为原始，直接赋值上文
            axs_small_temp.contourf(lon, lat, var_selmonth['mjja'][var_name][mod_name].values - var_selmonth['mjja'][var_name]['obs'].values,\
                levels = diff_ticks, cmap = cmap_diff, extend = 'both')

        plot_ind = plot_ind + 1

#----- add color bar-----

fig.colorbar(mcontourf_dict['mjja']['mean']['obs'], loc='b', width=0.1, extend = 'both',length = 0.9,\
ticklabelsize=5,labelsize = 7,ticks=mean_ticks[::2], title='near-surface air temperature ' + r"$[^\circ C$]")

fig.colorbar(mcontourf_dict['mjja']['mean']['vr'], loc='b', width=0.1, extend = 'both',length = 0.9,\
ticklabelsize=5,labelsize = 7,ticks=diff_ticks, title='bias ' + r"$[^\circ C$]")

# ----- format setting -----
axs.format(
abc=True,
abcloc = 'ul',
#----- 地图底图设置 -----
# reso = 'x-hi',
reso = 'med',
# coast = False,
coast = True,
coastlinewidth = 0.4,
borders = False,
lakes = False,
land  = False,
ocean = False,
# cartopyautoextent = True, 
# borderslinewidth=.5,
labels = False,
longrid  = True,
latgrid  = True,
#-----GEO axis-----
lonlim=(72, 136), latlim=(15, 55),
gridlabelsize = 5,
gridminor = True,
lonlocator = np.arange(70,142,10),
latlocator = np.arange(15,55+2,10),
lonminorlocator = np.arange(70,140+2,2),
latminorlocator = np.arange(15,55+2,2),
#-----line label-----
# linewidth = 0.5,
# suptitle="surface air temperature",
toplabels=('CN05.1', 'VR', 'RCM'),
leftlabels=('T2m','T2max','T2min'),
# leftlabelsize = 6.,
rc_kw = {'leftlabel.rotation':90.},
)

# 合并子图之后控制边界的labels绘制
axs[:-1,0].format(labels = True, lonlabels = False, latlabels = True)
axs[-1,1:].format(labels = True, lonlabels = True, latlabels = False)
axs[-1,0].format(labels = True, lonlabels = True, latlabels = True)

#----- save figure -----
fig.patch.set_facecolor('white')
fig.savefig('./output_pic/t2m_meanstates.2022.05.02.jpg', dpi=600, format = 'jpg', facecolor = 'white', bbox_inches='tight')